# Consortium Organisation DOI usage report

With the introduction of Consortium Members, it stopped being possible to easily get a DOI usage report that only includes our partners, as https://stats.datacite.org/ does not allow narrowing by that level of the hierarchy. Thankfully, all the information on the stats website is also available via the API, so we can build our own report.

## Contents

- [Setup](#Setup)
- [Fetch statistics](#Fetch-statistics)
- [Selecting Consortium Organisations](#Selecting-Consortium-Organisations)
- [Final report](#Final-report)
- [Save the results](#Save-the-results)

## Setup

First we import the python modules we'll need:

In [1]:
import aiohttp
import asyncio
import pandas as pd
import dcrest

We increase the number of rows that Pandas displays, so that we can see all the Consortium Organisations at once when we show the final table (we'll use `DataFrame.head` when we just want to check an intermediate result).

In [2]:
pd.set_option('display.max_rows', 120)

Configure the main parameters for the rest of the notebook. `CONSORTIUM_ID` can be changed to another consortium (lowercase).

In [3]:
CONSORTIUM_ID = "blco"

Create a HTTP client session: reusing this means we only have to set our options once, and it will also enable some optimisations that we don't need to care about ourselves. This is "asynchronous", which will be useful later when we need to make a large number of similar API calls in parallel, but it means we will need to use `await` each time we make an API request. We need to increase the timeout because DataCite API calls take about 10s to complete, and we reduce the number of simultaneous connections to avoid making too many at once and overloading the service (if we do this we will get blocked to preserve their service!).

In [4]:
session = aiohttp.ClientSession()
client = dcrest.DataCiteClient(session, test=False)

## Fetch statistics

We can fetch *all* the statistics with a single API call, and use `pandas.json_normalize` to convert the JSON response into a `DataFrame`. The endpoint to use is `/providers/totals`, because the API still refers to Direct Member and Consortium Organisation accounts as "providers". We use `DataFrame.set_index` to use the provider ID as the index column instead of the sequential numbers generated by default.

In [5]:
totals_data = await client.providers.totals()
totals = pd.json_normalize(totals_data) \
    .set_index('id')
totals.head()

,title,count,states,temporal.this_month,temporal.this_year,temporal.last_year,temporal.two_years_ago
id,,,,,,,
cern,CERN - European Organization for Nuclear Research,3377521,"[{'id': 'findable', 'title': 'Findable', 'coun...","[{'id': '2021', 'title': '2021', 'count': 77622}]","[{'id': '2021', 'title': '2021', 'count': 1065...","[{'id': '2020', 'title': '2020', 'count': 6554...","[{'id': '2019', 'title': '2019', 'count': 8340..."
figshare,figshare,2491795,"[{'id': 'findable', 'title': 'Findable', 'coun...","[{'id': '2021', 'title': '2021', 'count': 30489}]","[{'id': '2021', 'title': '2021', 'count': 5014...","[{'id': '2020', 'title': '2020', 'count': 4497...","[{'id': '2019', 'title': '2019', 'count': 3862..."
tawj,University of Tartu,2388849,"[{'id': 'findable', 'title': 'Findable', 'coun...","[{'id': '2021', 'title': '2021', 'count': 25}]","[{'id': '2021', 'title': '2021', 'count': 1676...","[{'id': '2020', 'title': '2020', 'count': 21431}]","[{'id': '2019', 'title': '2019', 'count': 11704}]"
stdp,ETH Zurich,2004608,"[{'id': 'findable', 'title': 'Findable', 'coun...","[{'id': '2021', 'title': '2021', 'count': 27444}]","[{'id': '2021', 'title': '2021', 'count': 2175...","[{'id': '2020', 'title': '2020', 'count': 1182...","[{'id': '2019', 'title': '2019', 'count': 1814..."
sage,SAGE Publishing,1798917,"[{'id': 'registered', 'title': 'Registered', '...","[{'id': '2021', 'title': '2021', 'count': 1306}]","[{'id': '2021', 'title': '2021', 'count': 87399}]","[{'id': '2020', 'title': '2020', 'count': 12728}]","[{'id': '2019', 'title': '2019', 'count': 13019}]"


Because of the structure of the returned JSON, we need to do a bit of extra work to extract the counts into columns on their own. We do this with `DataFrame.apply`, which runs a function (that we've defined) on each row of the dataset:

In [6]:
def extract_counts(row):
    result = {}
    for st in row.states:
        result[st['id']] = st['count']
    for period in ['this_year', 'last_year', 'two_years_ago']:
        data = row['temporal.' + period][0]
        result['count.' + data['id']] = data['count']
       
    return result

totals = totals[['title', 'count']].join(totals.apply(extract_counts, axis=1, result_type='expand'))
totals.head()

,title,count,findable,registered,count.2021,count.2020,count.2019
id,,,,,,,
cern,CERN - European Organization for Nuclear Research,3377521,2937247.0,440274.0,1065432.0,655475.0,834026.0
figshare,figshare,2491795,2349053.0,142742.0,501475.0,449750.0,386276.0
tawj,University of Tartu,2388849,2388833.0,16.0,1676671.0,21431.0,11704.0
stdp,ETH Zurich,2004608,2004551.0,57.0,217540.0,118294.0,181454.0
sage,SAGE Publishing,1798917,333723.0,1465194.0,87399.0,12728.0,13019.0


That's much better! We still have *all* the providers though, so now we need to select only those in our consortium.

## Selecting Consortium Organisations

The COs under a given Consortium Lead are available as part of the Lead's provider record, so we fetch that and make a Python set containing only the IDs of the Consortium Organisations, and use that to select only the totals we're interested in.

In [7]:
provider_data = await client.providers.get(CONSORTIUM_ID)
co_ids = {x['id'] for x in provider_data.relationships.consortiumOrganizations.data}
co_totals = totals[totals.index.isin(co_ids)]
co_totals.head()

,title,count,findable,registered,count.2021,count.2020,count.2019
id,,,,,,,
urks,Imperial College London,216289,210979.0,5310.0,2898.0,2813.0,8254.0
mqkk,Archaeology Data Service,90057,90036.0,21.0,6804.0,25822.0,7430.0
lpsw,University of Cambridge,76474,76350.0,124.0,16613.0,15278.0,12462.0
jhwt,Digital Repository of Ireland,59526,59525.0,1.0,8144.0,17495.0,5676.0
duqf,Science and Technology Facilities Council,18771,18766.0,5.0,935.0,2110.0,5796.0


The `/providers/totals` endpoint only returns a row for providers that have at least one DOI, but we also want to see which of our COs have 0 DOIs, so we will need to add these manually; reindexing the dataset with the full list of CO identifiers will add blank rows for these.

In [8]:
co_totals = co_totals.reindex(co_ids)

We only have the IDs for these blank rows, so we have to make a separate API call for each to fetch its name. We will use Python's `async` feature to allow us to do many of these in parallel: we build a list of the tasks, and then use `asyncio.gather` to handle actually running them and collecting the results.

In [9]:
async def get_title(org_id, client):
    try:
        data = await client.providers.get(org_id)
        title = data.attributes.name
    except aiohttp.ClientResponseError:
        title = '<deleted>'
        
    return {'id': org_id, 'title': title}

get_title_tasks = [get_title(org_id, client)
                   for org_id
                   in co_totals[co_totals.title.isna()].index]
extra_names = await asyncio.gather(*get_title_tasks)

extra_names = pd.DataFrame(extra_names).set_index('id').title

Now we can update our dataset with these extra names, and finally fill the remaining blank cells with 0 to give our final report.

In [10]:
co_totals['title'].update(extra_names)
co_totals = co_totals.fillna(0)

## Final report

In [11]:
co_totals

,title,count,findable,registered,count.2021,count.2020,count.2019
id,,,,,,,
tnae,Oxford Brookes University,1356.0,1355.0,1.0,237.0,226.0,209.0
urks,Imperial College London,216289.0,210979.0,5310.0,2898.0,2813.0,8254.0
eolj,University of Kent,1269.0,1269.0,0.0,648.0,580.0,23.0
rcnw,De Montfort University,649.0,549.0,100.0,162.0,302.0,182.0
qsee,Royal College of Music,76.0,76.0,0.0,36.0,12.0,28.0
xfir,Durham University,332.0,332.0,0.0,46.0,46.0,67.0
kpix,Edinburgh Napier University,111.0,105.0,6.0,32.0,63.0,7.0
gfww,Goldsmiths University of London,2055.0,2055.0,0.0,281.0,379.0,920.0
aevn,University of Stirling,0.0,0.0,0.0,0.0,0.0,0.0


## Save the results

As a final step, we'll export the results as an Excel spreadsheet for use elsewhere:

In [12]:
import datetime as dt

today = dt.datetime.now()
co_totals.to_excel(f'{CONSORTIUM_ID}-totals-{today:%Y%m%d}.xlsx')